## Deep Learning Study
#### Week 5: Natural language processing
![image](https://github.com/HanyangTechAI/2023-Deep-Learning-Study/assets/44901828/4322fd77-739a-4bb4-9e78-4fdca35cde20)
- Huggingface에서 제공하는 API를 활용하여 다양한 NLP task를 수행할 수 있는 모델을 불러와 사용해 봅시다.
- Transformers의 pipeline API를 사용하면 원본 텍스트 토크나이징, 모델 입력, 출력 결과 디코딩 등의 과정을 end-to-end로 쉽게 처리할 수 있습니다.
- 이 노트북에서는 transformer 아키텍처가 처음 소개된 논문인 Attention is All You Need 에서 나온 것과 같은 sequence-to-sequence 기계번역 task를 수행해보도록 하겠습니다.

In [1]:
# 현재 시스템에 transformers와 datasets 라이브러리를 설치합니다.
!pip install transformers datasets --quiet
import torch, random
import numpy as np

# 재현을 위해 seed 고정
random.seed(42)
np.random.seed(42)

# GPU 사용 여부
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 15.7 MB/s eta 0:00:00


- Huggingface는 트랜스포머 모델을 쉽게 불러와 학습/추론을 할 수 있게 하는 **transformers**를 비롯하여 다양한 라이브러리를 제공하고 있습니다.
- Pipeline을 사용하기 위해서는 이미 누군가 특정 task에 대해 학습시킨 모델을 불러와 사용해야 합니다.
- [Huggingface model hub](https://huggingface.co/models)에서 언어, 모델 아키텍처, task별로 검색을 통해 필요한 모델을 찾을 수 있습니다.

In [2]:
from transformers import pipeline

# 한국어->영어 번역을 수행하도록 학습된 Seq2Seq 모델을 사용하여 pipeline에 적용합니다.
# 모델 정보: https://huggingface.co/circulus/kobart-trans-ko-en-v2/tree/main
translator = pipeline("translation", model="circulus/kobart-trans-ko-en-v2", device=device)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


![image](https://github.com/HanyangTechAI/2023-Deep-Learning-Study/assets/44901828/93364508-4271-46cb-972b-59aff0e8ffe7)
- pipeline을 사용하는 것은 매우 간단합니다. 미리 준비해 둔 pipeline 객체를 하나의 함수처럼 사용할 수 있습니다.
- 번역의 대상인 text를 파라미터로 입력해주면 내장되어 있는 처리 과정이 수행된 결과가 dictionary 형태로 반환됩니다.
- 한번에 여러개의 입력을 batch로 사용하여 결과를 얻을 수 있습니다.

In [3]:
text = "오늘은 참 날씨가 좋아."

result = translator(text, max_length=64)
print("단일 입력 결과:", result)

texts = [
    "오늘 저녁에 뭐 먹지?",
    "기분이 정말 좋아!",
    "과제하기 너무 싫다...",
]
result = translator(texts, max_length=64)
print("다중 입력 결과:", result)

단일 입력 결과: [{'translation_text': 'The weather is so nice today'}]
다중 입력 결과: [{'translation_text': 'What are we having for dinner tonight'}, {'translation_text': 'I feel really good'}, {'translation_text': 'I hate the assignment'}]


- Huggingface는 다양한 언어와 task를 사용하는 데이터셋을 쉽게 사용할 수 있도록 하는 **dataset** 라이브러리를 제공합니다.
- Huggingface는 model hub와 마찬가지로 다양한 데이터셋을 찾을 수 있는 [dataset hub](https://huggingface.co/datasets)도 운영하고 있습니다.
- 네이버 뉴스 기사를 저장해둔 텍스트 데이터셋을 불러와서, 대량으로 영어로 번역하는 작업을 수행해 보겠습니다.

In [4]:
from datasets import load_dataset

# 네이버 뉴스 요약 데이터셋 다운로드
# 데이터셋 주소
naver_news = load_dataset("daekeun-ml/naver-news-summarization-ko")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/daekeun-ml___csv/daekeun-ml--naver-news-summarization-ko-884ccea06154613b/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

- 다운로드한 데이터셋을 살펴보면, train/validation/test 세 가지의 split으로 구성되어 있는 것을 볼 수 있습니다.
- test split을 선택해보면, 각각의 데이터셋은 table 형태로 구성되어 있고, 날짜, 제목, 본문, 요약문 등 다양한 column이 존재하는 것을 알 수 있습니다.
- 데이터셋에서 뉴스 제목을 10개 정도만 추출하여 영어로 번역해보도록 하겠습니다. 

In [5]:
print("전체 데이터셋:\n", naver_news)
print("Test split:\n", naver_news["test"])

news_titles = naver_news["test"]["title"][:10]
print("뉴스 제목:")
for title in news_titles:
  print(title)

전체 데이터셋:
 DatasetDict({
    train: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 22194
    })
    validation: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2466
    })
    test: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2740
    })
})
Test split:
 Dataset({
    features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
    num_rows: 2740
})
뉴스 제목:
아이트로닉스 차량용 복합기능형 졸음 방지 단말기 특허 출원
뉴스페이스로 가는 한국… 우주기업 성장할 기술·터전 만든다
SP500 올 상반기 21%↓…50년래 최악 월가월부
풀무원 알래스칸 명태살 ‘볼카츠’ 출시
일동제약 생산본부장에 강덕원 부사장 영입
강릉커피산업 발전·애로기술 해결 의기투합…출연기관 업무협약
약세장에 펀드 회전율 5년새 최저…운용기관 방어적 운용 영향
코오롱 수소 밸류체인 플랫폼 구축… 생산부터 발전까지 원스톱
에너지정책 유턴  재생에너지 대신 원전으로
산업 디지털전환법 오늘부터 시행…“기업 간 협업 프로젝트 지원 확대”


- 미리 준비해둔 한국어-영어 번역 pipeline에 추출한 뉴스 제목을 입력한 뒤 출력 결과를 list 형태로 바꾸어 보겠습니다.
- 뉴스 제목 데이터가 꽤 많기 때문에, 시간이 조금 걸릴 수 있습니다.

In [6]:
result = translator(news_titles, max_length=128)
english_titles = [result_dict["translation_text"] for result_dict in result]
print("번역 결과:")
for title in english_titles:
  print(title)

번역 결과:
A patent application for a complex functional drowsiness prevention terminal for Itronics vehicles
Korea going to the newsface... Space companies make technologies and technologies to grow
SP500 21 less in the first half of this year the worst in the year of 50 years
Pulmuone Alaskan pollack strips Bolkatsu is released
Vice President Kang Deokwon was recruited to head the production division of Ildong Pharmaceutical
Gangneung Coffee Industry Development and Difficult Technology Resolution Agreement of Business Agreement of Funded Institutions
The floating rate of the fund is the lowest in five years in a bearish market and the impact of defensive operations of management institutions
Building a Kolon hydrogen value chain platform Onestop from production to development
Energy Policy Uturn to nuclear power instead of renewable energy
The Industrial Digital Transformation Act starting today and “expanding support for collaboration projects between companies”


- 번역 결과가 나쁘지 않죠?
- Transformer 모델은 기계 번역 외에도 다양한 task를 수행할 수 있습니다.
- Huggingface model hub와 dataset hub에서 다양한 모델과 데이터를 조합하여 원하는 task를 수행해보시면 좋겠습니다.
- 아래 코드는 pipeline이 실행되는 내부 과정을 재현한 코드입니다. 토크나이징, 모델 입력 및 출력, 원본 형식(텍스트)로 디코딩 등의 과정이 구현되어 있습니다.

In [14]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# 모델, 토크나이저 다운로드
tokenizer = AutoTokenizer.from_pretrained("circulus/kobart-trans-ko-en-v2")
model = AutoModelForSeq2SeqLM.from_pretrained("circulus/kobart-trans-ko-en-v2").to(device)

# 입력 전처리
text = "오늘은 날씨가 참 좋아"
tokenized_text = tokenizer.encode(text, return_tensors="pt").to(device)
generated_tokens = model.generate(tokenized_text, max_length=128)
decoded_text = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)

print("원본 입력:\n", text)
print("\nTokenization 결과:\n", tokenized_text)
print("\n모델이 입력을 바탕으로 생성한 토큰:\n", generated_tokens)
print("\n텍스트 형태로 디코딩한 결과:\n", decoded_text)


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


원본 입력:
 오늘은 날씨가 참 좋아

Tokenization 결과:
 tensor([[22619, 27263, 14214, 15339]], device='cuda:0')

모델이 입력을 바탕으로 생성한 토큰:
 tensor([[    1, 29672, 20676, 26502, 16160, 14879,  1700, 15868, 18482,   310,
         21514, 16968,   300, 18090, 20223, 21777,     1]], device='cuda:0')

텍스트 형태로 디코딩한 결과:
 The weather is so nice today


![image](https://github.com/HanyangTechAI/2023-Deep-Learning-Study/assets/44901828/db54221f-0574-47bc-a6e8-66be2a3c732a)
- pipeline은 NLP task 뿐만 아니라 vision, voice, multimodal task 역시 지원합니다.
- 아래 task 목록과 [model hub](https://huggingface.co/models), [dataset hub](https://huggingface.co/datasets)을 참조하여 transformer 모델을 활용한 나만의 파이프라인을 구축해 보세요.

#### Reference
- Tasks: https://huggingface.co/docs/hub/models-tasks
- Translation: https://huggingface.co/docs/transformers/tasks/translation
- Pipeline: https://huggingface.co/docs/transformers/main_classes/pipelines

In [24]:
available_tasks = ['audio-classification', 'automatic-speech-recognition', 'conversational', 'depth-estimation', 'document-question-answering', 'feature-extraction', 'fill-mask', 'image-classification', 'image-segmentation', 'image-to-text', 'mask-generation', 'ner', 'object-detection', 'question-answering', 'sentiment-analysis', 'summarization', 'table-question-answering', 'text-classification', 'text-generation', 'text2text-generation', 'token-classification', 'translation', 'video-classification', 'visual-question-answering', 'vqa', 'zero-shot-audio-classification', 'zero-shot-classification', 'zero-shot-image-classification', 'zero-shot-object-detection', 'translation_XX_to_YY']
print("Pipeline에서 지원하는 task 목록")
for task in available_tasks: print(task)

Pipeline에서 지원하는 task 목록
audio-classification
automatic-speech-recognition
conversational
depth-estimation
document-question-answering
feature-extraction
fill-mask
image-classification
image-segmentation
image-to-text
mask-generation
ner
object-detection
question-answering
sentiment-analysis
summarization
table-question-answering
text-classification
text-generation
text2text-generation
token-classification
translation
video-classification
visual-question-answering
vqa
zero-shot-audio-classification
zero-shot-classification
zero-shot-image-classification
zero-shot-object-detection
translation_XX_to_YY
